## Hierarchical modularity in multiple cortical areas

We will collect neurons from several cortical areas to get the *functional connectivity* and perform the *dynamical analysis*.     
This will address three relevant points:
- Does the dynamic analysis hold at **higher temporal resolution**?
- Do **all areas of cortex** show attractor dynamics?
- Are population events only a **side-effect of behavior** (locomotion, whisker pad, pupil)?

To do all this, we analyse the [data](https://janelia.figshare.com/articles/dataset/Eight-probe_Neuropixels_recordings_during_spontaneous_behaviors/7739750/4) by [Stringer et al. 2019](science.org/doi/10.1126/science.aav7893).   
Eight-probe Neuropixels recordings in three mice during spontaneous activity.

In [2]:
from platform import python_version
print(python_version())

from builtins import exec
exec(open("./imports_functions.py").read())

%matplotlib inline

3.10.4


In [ ]:
if not os.path.exists("stringer/7739750.zip"):
    print("Downloading neuropixel data ...")
    resp = wget.download("https://janelia.figshare.com/ndownloader/articles/7739750/versions/4", "stringer/7739750.zip")
    print("... Done: "+resp)

# unzip downloaded folder
# unzip spks.zip and faces.zip

This analysis is based on the file `ephysLoad.m`.

Each "spks" is a structure of length 8, where each entry is a different probe (these probes were recorded simultaneously). It contains the spike times (in seconds, e.g. 4048.44929626 sec = 4048449.29626 ms (?kHz sampling)), the cluster identity of each spike (its cell), and the height of each cluster on the probe.

The location of each site on the probe in microns in the Allen CCF framework is given in "ccfCoords". The brain area for each site is in "borders" as a function of the height of the site. 

In [10]:
from scipy import io

probeLoc = io.loadmat('stringer/7739750/probeLocations.mat')
probeBorders = io.loadmat('stringer/7739750/probeBorders.mat', squeeze_me=True)

mouse_names = ['Krebs','Waksman','Robbins']
cortical_areas = ['FrCtx','FrMoCtx','SomMoCtx','SSCtx','V1','V2','RSP']

# first count the cells you want to take with this structure
# then think on how you want to store the spikes... compatible with the dynamical_analysis
area_spiketrains = {
    'Krebs' : {'FrCtx':[], 'FrMoCtx':[], 'SomMoCtx':[], 'SSCtx':[], 'V1':[], 'V2':[], 'RSP':[]},
    'Waksman' : {'FrCtx':[], 'FrMoCtx':[], 'SomMoCtx':[], 'SSCtx':[], 'V1':[], 'V2':[], 'RSP':[]},
    'Robbins' : {'FrCtx':[], 'FrMoCtx':[], 'SomMoCtx':[], 'SSCtx':[], 'V1':[], 'V2':[], 'RSP':[]}
}

# start of spontaneous activity in each mouse (in seconds)
tstart = [3811, 3633, 3323] # what is there before?

for imouse in range(len(mouse_names)):
    print(mouse_names[imouse])
    
    spks = io.loadmat('stringer/7739750/spks/spks%s_Feb18.mat'%mouse_names[imouse], squeeze_me=True)

    # probe k
    # k = 7
    for k in range(8):
        print("probe",k)
    
        # spike times (in seconds)
        st = spks['spks'][k][0]
        # print(st)
        print(len(st))
        # clusters
        clu = spks['spks'][k][1]
        print("clusters (cells) of the spikes",len(np.unique(clu)))
        # cluster heights (in microns)
        # (see siteCoords to convert to site location)
        Wh = spks['spks'][k][2]

        # where is the probe in the brain (consolidated labels)
        # borders are in microns
        # use Wh to determine which clusters are in which brain region
        borders = probeBorders['probeBorders'][imouse]['borders'][k]
        for j in range(len(borders)):
            b = borders[j]
            if b[2] not in cortical_areas:
                continue
            print('upper border %d um, lower border %d um, area %s'%(b[0],b[1],b[2]))
            wneurons = np.logical_and(Wh>=b[1], Wh<b[0])
            print(wneurons)
            print(clu[wneurons]) # not working
            nn = wneurons.sum()
            print('%d neurons in %s'%(nn,b[-1]))


        # # where is the probe in the brain (in microns)
        # ccfCoords = probeLoc['probeLocations'][0][imouse]['probe'][k][0]['ccfCoords']
        # # name of area in Allen ontology by site on electrode
        # ccfNames = probeLoc['probeLocations'][0][imouse]['probe'][k][0]['ccfOntology']
        # # coordinates of each site on the electrode
        # siteCoords = probeLoc['probeLocations'][0][imouse]['probe'][k][0]['siteCoords']

        # # processed faces, to correlate behavior
        # # The behavioral file is the processed version of a mouse face movie (time x pixels x pixels). 
        # faces = io.loadmat('stringer/7739750/faces/%s_face_proc.mat'%mouse_names[imouse], squeeze_me=True)
        # motSVD = faces['motionSVD']
        # video_timestamps = faces['times']

        print()

Krebs
probe 0
606058
clusters (cells) of the spikes 127
upper border 4000 um, lower border 1100 um, area FrMoCtx
[False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False  True  True  True  True False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False  True False
 False False False False False False False]


IndexError: boolean index did not match indexed array along dimension 0; dimension is 606058 but corresponding boolean dimension is 127